# Network Setup

### Create the Grid Network

In [5]:
import networkx as nx
G = nx.grid_2d_graph(4, 4)

# transform the grid to edge-based
G_edge = nx.Graph()

# create nodes using edges
for e in list(G.edges()):
    G_edge.add_node(e)

for n1 in G_edge.nodes():
    for n2 in G_edge.nodes():
        if not n1 == n2:
            if n1[1] == n2[0]:
                G_edge.add_edge(n1, n2)
list(G_edge.nodes())

[((0, 0), (1, 0)),
 ((0, 0), (0, 1)),
 ((0, 1), (1, 1)),
 ((0, 1), (0, 2)),
 ((0, 2), (1, 2)),
 ((0, 2), (0, 3)),
 ((0, 3), (1, 3)),
 ((1, 0), (2, 0)),
 ((1, 0), (1, 1)),
 ((1, 1), (2, 1)),
 ((1, 1), (1, 2)),
 ((1, 2), (2, 2)),
 ((1, 2), (1, 3)),
 ((1, 3), (2, 3)),
 ((2, 0), (3, 0)),
 ((2, 0), (2, 1)),
 ((2, 1), (3, 1)),
 ((2, 1), (2, 2)),
 ((2, 2), (3, 2)),
 ((2, 2), (2, 3)),
 ((2, 3), (3, 3)),
 ((3, 0), (3, 1)),
 ((3, 1), (3, 2)),
 ((3, 2), (3, 3))]

### Covariance Matrix

In [6]:
import numpy as np
from scipy.linalg import expm

# generate co-variance matrix from the Laplacian matrix
cov = expm(-nx.normalized_laplacian_matrix(G_edge).toarray())

### Parameters in Example 2,3

In [7]:
mu = 1
tau_squared = 0.2

### Route Data

In [8]:
# historical data
hist_route = [
                [((1,0), (1,1)), ((1,1),(2,1)), ((2,1),(3,1))],
                [((1,1), (1,2)), ((1,2),(2,2)), ((2,2),(3,2))],
                [((1,1), (1,2)), ((1,2),(1,3)), ((1,3),(2,3))],
                [((1,0), (1,1)), ((1,1),(1,2))],
                [((1,0), (1,1)), ((0,1),(1,1))],
                [((1,3), (2,3)), ((2,3),(3,3))]
             ]
hist_route_idx = []
for route in hist_route:
    route_idx = []
    for e in route:
        route_idx.append(list(G_edge.nodes()).index(e))
    hist_route_idx.append(route_idx)


# route to predict
route_to_predict = [8, 10]

# Example 2: Optimal Estimator

In [9]:
hist_route_mtx = np.zeros((len(hist_route), len(list(G_edge.nodes()))))
for i in range(len(hist_route_idx)):
    for s in hist_route_idx[i]:
        hist_route_mtx[i,s] = 1

M = 0
for route in hist_route:
    M += len(route)

# populate matrix Phi
Phi = np.zeros((M,M))
init = 0
for route_idx in hist_route_idx:
    Phi[init:init+len(route_idx), init:init+len(route_idx)] = cov[route_idx,:][:,route_idx]
    init += len(route_idx)

# populate matrix X
U = np.zeros((M,len(G_edge.nodes())))
count = 0
for route_idx in hist_route_idx:
    for e in route_idx:
        U[count, e] = 1
        count += 1

# calculate matrix Q
Q = np.matmul(np.matmul(np.transpose(U),np.linalg.inv(Phi)), U) + np.diag((1/tau_squared)*np.ones(len(G_edge.nodes())))

# e_y
e_y = np.zeros(len(G_edge.nodes()))
e_y[route_to_predict] = 1

# E_y
E_y = np.matmul(np.reshape(e_y, [24,1]), np.transpose(np.reshape(e_y, [24,1])))

print('optimal weights:',np.transpose(e_y)@np.linalg.inv(Q)@np.transpose(U)@np.linalg.inv(Phi))
print('optimal intercept:', mu/tau_squared * np.transpose(e_y)@np.linalg.inv(Q)@np.ones(len(G_edge.nodes())))

# calculate the optimal risk
temp = E_y + np.transpose(U)@np.linalg.inv(Phi)@U@np.linalg.inv(Q)@E_y@np.linalg.inv(Q)@np.transpose(U)@np.linalg.inv(Phi)@U - 2*E_y@np.linalg.inv(Q)@np.transpose(U)@np.linalg.inv(Phi)@U
optimal_risk = np.trace(np.diag(tau_squared*np.ones(len(G_edge.nodes())))@temp) + np.trace(np.linalg.inv(Phi)@U@np.linalg.inv(Q)@E_y@np.linalg.inv(Q)@np.transpose(U))

print('optimal risk:',optimal_risk)

variance_optimal_risk =  np.trace(np.linalg.inv(Phi)@U@np.linalg.inv(Q)@E_y@np.linalg.inv(Q)@np.transpose(U))
print('expected variance:',variance_optimal_risk)

bias_optimal_risk = np.trace(np.diag(tau_squared*np.ones(len(G_edge.nodes())))@temp)
print('expected squared bias:',bias_optimal_risk)

optimal weights: [ 2.10594179e-01 -3.99067236e-02  1.81394762e-03  2.07325801e-01
 -3.41917565e-02  1.59055627e-03  2.10021317e-01 -3.98899563e-02
  2.32940749e-03  1.57180065e-01  1.55878021e-01  2.00550262e-01
 -1.04021947e-02 -7.63048957e-04  2.00434654e-04]
optimal intercept: 0.9776696879718729
optimal risk: 0.17169007079118165
expected variance: 0.09710009795107723
expected squared bias: 0.07458997284010441


# Example 3: Segment-, Generalized Segment- and Route-Based Estimators

In [10]:
# sample size matrix N_{st}
N = np.zeros((len(route_to_predict), len(route_to_predict)))
for i in range(len(route_to_predict)):
    for j in range(len(route_to_predict)):
        N[i,j] = np.sum(hist_route_mtx[:,route_to_predict[i]]*hist_route_mtx[:,route_to_predict[j]],0)

# calculate the optimal weights of segment-based estimator
A = np.zeros((len(route_to_predict), len(route_to_predict)))
for i in range(len(route_to_predict)):
    for j in range(len(route_to_predict)):
        A[i,j] = cov[route_to_predict[i], route_to_predict[j]]*N[i,j]/(N[i,i]*N[j,j])
for i in range(len(route_to_predict)):
    A[i,i] += tau_squared
b = np.ones(len(route_to_predict))*tau_squared
phi_seg = np.linalg.inv(A)@b
print('optimal weights of segment-based estimator:',phi_seg)

# risk of the optimal segment-based estimator
risk_seg = 0
for i in range(len(route_to_predict)):
    for j in range(len(route_to_predict)):
        risk_seg += (N[i,j]/(N[i,i]*N[j,j]))*phi_seg[i]*phi_seg[j]*cov[route_to_predict[i], route_to_predict[j]]
for i in range(len(route_to_predict)):
    risk_seg += (1-phi_seg[i])**2 * tau_squared
print('risk of the optimal segment-based estimator:',risk_seg,"\n")


# risk of generalized segment-based estimator
total_cov = 0
for i in range(len(route_to_predict)):
    for j in range(len(route_to_predict)):
        total_cov += cov[route_to_predict[i], route_to_predict[j]]
phi_g_seg = (len(route_to_predict)*tau_squared)/(len(route_to_predict)*tau_squared + total_cov)
print('optimal weights of the generalized seg-based estimator:',phi_g_seg)
risk_g_seg = (phi_g_seg**2)*total_cov + ((1 - phi_g_seg)**2)*len(route_to_predict)*tau_squared
print('risk of the generalized segment-based estimator:',risk_g_seg,"\n")

# risk of optimal route-based estimator
temp1 = [8,10]
temp2 = [8,2]
total_cov = 0
for i in temp1:
    for j in temp1:
        total_cov += cov[i, j]
for i in temp2:
    for j in temp2:
        total_cov += cov[i, j]
phi_route = (3*tau_squared)/(2*tau_squared + (1/2)*tau_squared + (1/2)*tau_squared + total_cov/2)
print('optimal weights of route-based estimator:',phi_route)
risk_route = (phi_route/2)**2 * tau_squared + (1-phi_route/2)**2 * tau_squared + (1-phi_route)**2 * tau_squared + ((phi_route/2)**2 * total_cov)
print('risk of the route-based estimator:', risk_route)

optimal weights of segment-based estimator: [0.55986267 0.56193637]
risk of the optimal segment-based estimator: 0.17564019103497866 

optimal weights of the generalized seg-based estimator: 0.26722920855173377
risk of the generalized segment-based estimator: 0.29310831657930647 

optimal weights of route-based estimator: 0.37188459376336797
risk of the route-based estimator: 0.28843462187098967


# Example 4: Negative Covariance

In [29]:
# new parameters
tau_squared = 1
cov_neg = np.array([[1,-0.9],[-0.9,1]])

# sample size matrix N_{st}
N = np.zeros((len(route_to_predict), len(route_to_predict)))
for i in range(len(route_to_predict)):
    for j in range(len(route_to_predict)):
        N[i,j] = np.sum(hist_route_mtx[:,route_to_predict[i]]*hist_route_mtx[:,route_to_predict[j]],0)

# calculate optimal weights of segment-based estimator
A = np.zeros((len(route_to_predict), len(route_to_predict)))
for i in range(len(route_to_predict)):
    for j in range(len(route_to_predict)):
        A[i,j] = cov_neg[i,j]*N[i,j]/(N[i,i]*N[j,j])
for i in range(len(route_to_predict)):
    A[i,i] += tau_squared
b = np.ones(len(route_to_predict))*tau_squared
phi_seg = np.linalg.inv(A)@b
print('optimal weights of segment-based estimator:',phi_seg)

# risk of the optimal segment-based estimator
risk_seg = 0
for i in range(len(route_to_predict)):
    for j in range(len(route_to_predict)):
        risk_seg += (N[i,j]/(N[i,i]*N[j,j]))*phi_seg[i]*phi_seg[j]*cov_neg[i, j]
for i in range(len(route_to_predict)):
    risk_seg += (1-phi_seg[i])**2 * tau_squared
print('risk of the optimal segment-based estimator:',risk_seg,'\n')


# risk of the optimal route-based estimator
total_cov = 0
for i in range(len(route_to_predict)):
    for j in range(len(route_to_predict)):
        total_cov += cov_neg[i, j]

phi_route = (len(route_to_predict)*tau_squared)/(len(route_to_predict)*tau_squared + total_cov)
print('optimal weights of route-based estimator:',phi_route)
risk_route = (phi_route**2)*total_cov + ((1 - phi_route)**2)*len(route_to_predict)*tau_squared

print('risk of the route-based estimator:', risk_route)

optimal weights of segment-based estimator: [0.81081081 0.81081081]
risk of the optimal segment-based estimator: 0.37837837837837834 

optimal weights of route-based estimator: 0.9090909090909091
risk of the route-based estimator: 0.18181818181818177


# Example 5

In [11]:
# new route to predict
route_to_predict = [8, 10, 11]
cov_new = np.array([[0.1, 1.0, 0],
                    [1.0, 10.0, 0],
                    [0, 0, 10.0]])
tau_squared = 1

# sample size
N = np.array([[1, 1, 0], [1, 2, 1], [0, 1, 1]])

# calculate the optimal weights of segment-based estimator
A = np.zeros((len(route_to_predict), len(route_to_predict)))
for i in range(len(route_to_predict)):
    for j in range(len(route_to_predict)):
        A[i,j] = cov_new[i,j] * N[i,j]/(N[i,i] * N[j,j])
for i in range(len(route_to_predict)):
    A[i,i] += tau_squared
b = np.ones(len(route_to_predict))*tau_squared
phi_seg = np.linalg.inv(A)@b
print('optimal weights of segment-based estimator:',phi_seg)

# risk of the optimal segment-based estimator
risk_seg = 0
for i in range(len(route_to_predict)):
    for j in range(len(route_to_predict)):
        risk_seg += (N[i,j]/(N[i,i]*N[j,j]))*phi_seg[i]*phi_seg[j]*cov_new[i,j]

for i in range(len(route_to_predict)):
    risk_seg += (1-phi_seg[i])**2 * tau_squared
print('risk of the optimal segment-based estimator:',risk_seg,'\n')


# generalized segment-based estimator
A_part = np.zeros((2,2))
A_part[0,0] = np.sum(cov_new[0,0])+ tau_squared
A_part[1,1] = np.sum(cov_new[1:3,1:3])+ 2 * tau_squared
b = np.ones(2)
b[0] = tau_squared
b[1] = 2*tau_squared
phi_g_seg = np.linalg.inv(A_part)@b
print('optimal weights of generalized segment-based estimator:',phi_g_seg)


risk_g_seg = 0
risk_g_seg += phi_g_seg[0] * phi_g_seg[0] * np.sum(cov_new[0,0])
risk_g_seg += phi_g_seg[1] * phi_g_seg[1] * np.sum(cov_new[1:3,1:3])

risk_g_seg += ((1-phi_g_seg[0])**2) * tau_squared
risk_g_seg += ((1-phi_g_seg[1])**2) * 2 * tau_squared
print('risk of generalized segment-based estimator:',risk_g_seg)

optimal weights of segment-based estimator: [0.86614173 0.09448819 0.09090909]
risk of the optimal segment-based estimator: 1.9484609878310664 

optimal weights of generalized segment-based estimator: [0.90909091 0.09090909]
risk of generalized segment-based estimator: 1.909090909090909
